In [1]:
import numpy as np, pandas as pd

In [2]:
fee_detail = pd.read_csv('../data/fee_detail.csv',low_memory=False)
df_train = pd.read_csv('../data/df_train_from mail.csv',low_memory=False)

In [3]:
df_id_train = pd.read_csv('../data/df_id_train.csv',low_memory=False,header=None)

## 数据预处理

### 数据剔除

In [4]:
#df_train中只有一个值的column
del_columns1 = df_train.columns[df_train.nunique() == 1]
del_columns1

Index(['药品费拒付金额', '检查费拒付金额', '治疗费拒付金额', '手术费拒付金额', '床位费拒付金额', '医用材料费拒付金额',
       '输全血申报金额', '成分输血自费金额', '成分输血拒付金额', '其它拒付金额', '一次性医用材料自费金额',
       '输全血按比例自负金额', '统筹拒付金额', '双笔退费标识', '住院天数', '非典补助补助金额', '家床起付线剩余'],
      dtype='object')

In [5]:
#df_train中缺失值大于一半的column
del_columns2 = df_train.columns[df_train.isnull().sum() >= 0.9*df_train.shape[0]]
del_columns2

Index(['一次性医用材料拒付金额', '农民工医疗救助计算金额', '公务员医疗补助基金支付金额', '残疾军人医疗补助基金支付金额'], dtype='object')

In [6]:
#fee_detail中只有一个值的column
del_columns3 = fee_detail.columns[fee_detail.nunique() == 1]
del_columns3

Index([], dtype='object')

In [7]:
#fee_detail中只有一个值的column
del_columns4 = fee_detail.columns[fee_detail.isnull().sum() >= 0.9*fee_detail.shape[0]]
del_columns4

Index(['拒付原因编码', '拒付原因'], dtype='object')

In [8]:
del_columns1_all = list(set(del_columns1)|set(del_columns2))
del_columns2_all = list(set(del_columns3)|set(del_columns4))

In [9]:
df_train = df_train.drop(columns=del_columns1_all)

In [10]:
fee_detail = fee_detail.drop(columns=del_columns2_all)

### 缺失值填补

两者缺失值数据很少 仅1000条数据(不足1000/180w)，可以直接删除

In [11]:
#df_train = df_train.dropna()
#fee_detail = fee_detail.dropna()

### One-hot 编码

统计并保留每个离散型变量出现频率最高的10种(不足10则全部)

In [12]:
df_train[df_train.columns[df_train.dtypes == 'object']]

,顺序号,交易时间,住院开始时间,住院终止时间,申报受理时间,出院诊断病种名称,操作时间
0,MT01801607015822812,2016-07-01,01-7月 -16,01-7月 -16,01-7月 -16,挂号,01-7月 -16
1,MT01431607015822815,2016-07-01,01-7月 -16,01-7月 -16,01-7月 -16,挂号,01-7月 -16
2,MT01101607015822817,2016-07-01,01-7月 -16,01-7月 -16,01-7月 -16,挂号,01-7月 -16
3,MT01101607015822838,2016-07-01,01-7月 -16,01-7月 -16,01-7月 -16,挂号,01-7月 -16
4,MT01101607015822840,2016-07-01,01-7月 -16,01-7月 -16,01-7月 -16,挂号,01-7月 -16
...,...,...,...,...,...,...,...
1830381,MT01951612166757345,2016-12-16,16-12月-16,NaN,NaN,NaN,16-12月-16
1830382,MT01051612156604530,2016-12-15,15-12月-16,NaN,NaN,NaN,15-12月-16
1830383,MT01801612186871163,2016-12-18,18-12月-16,NaN,NaN,NaN,18-12月-16
1830384,MT01801612146514320,2016-12-14,14-12月-16,NaN,NaN,NaN,14-12月-16


In [13]:
fee_detail[fee_detail.columns[fee_detail.dtypes == 'object']]

,顺序号,三目服务项目名称,三目医院服务项目名称,剂型,规格,费用发生时间
0,MT13061601133880864,普通门诊挂号费---三级医院,挂号,,,2016-01-13
1,MT13061601133880864,NaN,挂号,,,2016-01-13
2,MT13061601154123666,NaN,挂号,,,2016-01-15
3,MT13061601154123666,普通门诊挂号费---三级医院,挂号,,,2016-01-15
4,MT13061602298982056,普通门诊挂号费---三级医院,挂号,,,2016-02-29
...,...,...,...,...,...,...
6532896,MT01501612318391547,NaN,NaN,,,2016-12-31
6532897,MT01501612318391547,NaN,NaN,注射剂,300单位,2016-12-31
6532898,MT01581612318446005,NaN,NaN,滴丸,27mg*180,2016-12-31
6532899,MT05871612318458397,NaN,NaN,,,2016-12-31


In [14]:
cat_columns1 = ['出院诊断病种名称']
cat_columns2 = ['三目服务项目名称','三目医院服务项目名称']

In [15]:
#出现频率前十的诊断病种
top_10_ZDBZ = list(df_train[cat_columns1].value_counts()[:10].index)
top_10_ZDBZ = list(map(lambda x: x[0],top_10_ZDBZ))
top_10_ZDBZ

['挂号',
 '门特挂号',
 '糖尿病',
 '偏瘫',
 '精神病',
 '肺心病',
 '慢性肺源性心脏病;冠心病;高血压;过敏性鼻炎',
 '慢性肺源性心脏病;高血压;冠心病;过敏性鼻炎',
 '糖尿病  ',
 '糖尿病（门特）']

In [16]:
#出现频率前十的三目服务
top_10_SMFW = list(fee_detail[cat_columns2[0]].value_counts()[:10].index)
#top_10_SMFW = list(map(lambda x: x[0],top_10_SMFW))
top_10_SMFW

['普通门诊挂号费---一级医院',
 '糖尿病门诊诊察费（门特加收）',
 '脑心通胶囊',
 '多导心电图检查自动分析',
 '通心络胶囊',
 '肾炎康复片',
 '速效救心丸',
 '依诺肝素钠注射液',
 '低分子量肝素钙注射液',
 '普通门诊挂号费---二级医院']

In [17]:
#出现频率前十的三目医院服务
top_10_SMYYFW = list(fee_detail[cat_columns2[1]].value_counts()[:10].index)
#top_10_SMYYFW = list(map(lambda x: x[0],top_10_SMYYFW))
top_10_SMYYFW

['普通门诊挂号费---一级医院',
 '糖尿病门诊诊察费（门特加收）',
 '糖尿病门诊诊察费',
 '挂号',
 '普通门诊挂号费',
 '脑心通胶囊',
 '三级医院-普通门诊挂号费',
 '通心络胶囊',
 '糖尿病门诊诊察费加收',
 '门诊诊查费糖尿病加收']

In [18]:
#其余的种类数据去除
df_train_new = df_train[df_train['出院诊断病种名称'].isin(top_10_ZDBZ)]
fee_detail_new = fee_detail[fee_detail['三目服务项目名称'].isin(top_10_SMFW)&fee_detail['三目医院服务项目名称'].isin(top_10_SMYYFW)]

In [19]:
df_train_new.shape

(156934, 48)

In [20]:
fee_detail_new.shape

(609717, 10)

In [21]:
df_train_new = pd.concat([df_train_new,pd.get_dummies(df_train_new['出院诊断病种名称'])],axis=1)
df_train_new.shape

(156934, 58)

In [22]:
fee_detail_new = pd.concat([fee_detail_new,pd.get_dummies(fee_detail_new[cat_columns2])],axis=1)
fee_detail_new.shape

(609717, 24)

In [23]:
fee_detail_new['顺序号'].nunique()

609659

## 特征构造

### 医院欺诈系数12

In [24]:
df_id_train

,0,1
0,352120001523108,1
1,352120001475556,0
2,352120003484886,0
3,352120002750505,0
4,352120001556755,0
...,...,...
19995,352120001911523,0
19996,352120003184869,0
19997,352120000762238,0
19998,352120002142283,0


In [25]:
#出院诊断病种名称

In [26]:
#个人编号与欺诈字典
person_to_fraud_dic = dict(zip(df_id_train[0],df_id_train[1]))

In [27]:
df_train.head()

,顺序号,个人编码,医院编码,药品费发生金额,贵重药品发生金额,中成药费发生金额,中草药费发生金额,药品费自费金额,药品费申报金额,检查费发生金额,...,住院开始时间,住院终止时间,申报受理时间,出院诊断病种名称,本次审批金额,补助审批金额,医疗救助医院申请,民政救助补助金额,城乡优抚补助金额,操作时间
0,MT01801607015822812,352120001523108,180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,01-7月 -16,01-7月 -16,01-7月 -16,挂号,3.0,0.6,0.0,0.48,0.0,01-7月 -16
1,MT01431607015822815,352120001475556,143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,01-7月 -16,01-7月 -16,01-7月 -16,挂号,12.5,NaN,0.0,0.00,0.0,01-7月 -16
2,MT01101607015822817,352120003484886,110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,01-7月 -16,01-7月 -16,01-7月 -16,挂号,2.5,NaN,0.0,0.00,0.0,01-7月 -16
3,MT01101607015822838,352120002750505,110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,01-7月 -16,01-7月 -16,01-7月 -16,挂号,15.0,NaN,0.0,0.00,0.0,01-7月 -16
4,MT01101607015822840,352120001556755,110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,01-7月 -16,01-7月 -16,01-7月 -16,挂号,2.5,NaN,0.0,0.00,0.0,01-7月 -16


In [28]:
is_fraud = list(map(lambda x: person_to_fraud_dic[x],df_train['个人编码']))

In [29]:
df_train.insert(2,'is_fraud',is_fraud)
df_train.insert(3,'all',np.ones(len(df_train)))

In [30]:
df_train.groupby('医院编码').sum()['is_fraud'].values/df_train.groupby('医院编码').sum()['all'].values

/var/folders/bl/s33lb64n5876ysbhd2h660tw0000gn/T/ipykernel_58632/1428009439.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_train.groupby('医院编码').sum()['is_fraud'].values/df_train.groupby('医院编码').sum()['all'].values
/var/folders/bl/s33lb64n5876ysbhd2h660tw0000gn/T/ipykernel_58632/1428009439.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_train.groupby('医院编码').sum()['is_fraud'].values/df_train.groupby('医院编码').sum()['all'].values


array([0.04817927, 0.        , 0.09044728, 0.0527765 , 0.02406471,
       0.        , 0.        , 0.03099059, 0.        , 0.00958773,
       0.01809084, 0.01254902, 0.        , 0.01564537, 0.20942239,
       0.02581369, 0.        , 0.        , 0.01868365, 0.00834703,
       0.02364532, 0.0157385 , 0.05784892, 0.04382658, 0.        ,
       0.08534851, 0.        , 0.00242424, 0.04576659, 0.05260389,
       0.00373333, 0.05405405, 0.04615385, 0.02941176, 0.027     ,
       0.11618257, 0.02488889, 0.        , 0.0490077 , 0.        ,
       0.06340956, 0.        , 0.        , 0.03407407, 0.05298013,
       0.        , 0.        , 0.04919953, 0.00918448, 0.10898929,
       0.        , 0.00988794, 0.00904496, 0.01535898, 0.01566952,
       0.        , 0.00876424, 0.0525    , 0.0205803 , 0.02538297,
       0.05803869, 0.00984252, 0.1641791 , 0.        , 0.06183442,
       0.0043611 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.11071429, 0.44577

In [31]:
#医院欺诈系数1
hospital_to_fraud_dict1 = dict(zip(df_train.groupby('医院编码').sum()['is_fraud'].index,\
        df_train.groupby('医院编码').sum()['is_fraud'].values/df_train.groupby('医院编码').sum()['all'].values))

/var/folders/bl/s33lb64n5876ysbhd2h660tw0000gn/T/ipykernel_58632/2883650926.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  hospital_to_fraud_dict1 = dict(zip(df_train.groupby('医院编码').sum()['is_fraud'].index,\
/var/folders/bl/s33lb64n5876ysbhd2h660tw0000gn/T/ipykernel_58632/2883650926.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_train.groupby('医院编码').sum()['is_fraud'].values/df_train.groupby('医院编码').sum()['all'].values))
/var/folders/bl/s33lb64n5876ysbhd2h660tw0000gn/T/ipykernel_58632/2883650926.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a fu

In [32]:
#申报金额的所有列
columns_sbje = list(filter(lambda x: '申报金额' in x, df_train.columns))
columns_sbje

['药品费申报金额',
 '检查费申报金额',
 '治疗费申报金额',
 '手术费申报金额',
 '床位费申报金额',
 '成分输血申报金额',
 '其它申报金额',
 '一次性医用材料申报金额']

In [33]:
#欺诈人员申报总金额
dic_tem2 = df_train[df_train['is_fraud'] == 1].groupby('医院编码').sum()[columns_sbje].sum(axis=1)

#医院总金额
dic_tem1 = df_train.groupby('医院编码').sum()[columns_sbje].sum(axis=1)

#补全
for key in dic_tem1.keys():
    if key not in dic_tem2.keys():
        dic_tem2[key] = 0

dic_tem2 = dic_tem2.sort_index()
#医院欺诈系数2
hospital_to_fraud_dict2 = (dic_tem2/dic_tem1).replace(np.nan,0)

/var/folders/bl/s33lb64n5876ysbhd2h660tw0000gn/T/ipykernel_58632/2130990860.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dic_tem2 = df_train[df_train['is_fraud'] == 1].groupby('医院编码').sum()[columns_sbje].sum(axis=1)
/var/folders/bl/s33lb64n5876ysbhd2h660tw0000gn/T/ipykernel_58632/2130990860.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dic_tem1 = df_train.groupby('医院编码').sum()[columns_sbje].sum(axis=1)


In [34]:
#个人编码对医院编码的dic
person_to_hospital_dic = dict(zip(df_train.drop_duplicates(['个人编码','医院编码'])['个人编码'],\
                                  df_train.drop_duplicates(['个人编码','医院编码'])['医院编码']))

In [35]:
qz1 = []
qz2 = []
for i in range(len(df_id_train)):
    try:
        q1 = hospital_to_fraud_dict1[person_to_hospital_dic[df_id_train[0][i]]]
        qz1.append(q1)
    except:
        qz1.append(np.nan)
        
    try:
        q2 = hospital_to_fraud_dict2[person_to_hospital_dic[df_id_train[0][i]]]
        qz2.append(q2)
    except:
        qz2.append(q2)

In [36]:
df_id_train.insert(2,'医院欺诈系数1',qz1)
df_id_train.insert(3,'医院欺诈系数2',qz2)

In [37]:
df_id_train

,0,1,医院欺诈系数1,医院欺诈系数2
0,352120001523108,1,0.207418,0.209154
1,352120001475556,0,0.053581,0.070967
2,352120003484886,0,0.074702,0.072067
3,352120002750505,0,0.003731,0.001094
4,352120001556755,0,0.000000,0.000000
...,...,...,...,...
19995,352120001911523,0,0.176705,0.191084
19996,352120003184869,0,0.017310,0.012591
19997,352120000762238,0,0.041901,0.040705
19998,352120002142283,0,0.134112,0.136299


### 单日同一医院就诊次数(有不同日期的取最大值)

In [38]:
df_id_train[0][0]

352120001523108

In [39]:
df_tem = df_train.groupby(['个人编码','操作时间']).count()

In [40]:
df_id_train

,0,1,医院欺诈系数1,医院欺诈系数2
0,352120001523108,1,0.207418,0.209154
1,352120001475556,0,0.053581,0.070967
2,352120003484886,0,0.074702,0.072067
3,352120002750505,0,0.003731,0.001094
4,352120001556755,0,0.000000,0.000000
...,...,...,...,...
19995,352120001911523,0,0.176705,0.191084
19996,352120003184869,0,0.017310,0.012591
19997,352120000762238,0,0.041901,0.040705
19998,352120002142283,0,0.134112,0.136299


In [41]:
one_day_counts = []
for i in range(len(df_id_train)):
    count = df_tem.loc[df_id_train[0][i]]['医院编码'].max()
    one_day_counts.append(count)

In [42]:
df_id_train.insert(4,'单日同一医院最大就诊次数',one_day_counts)

In [43]:
df_id_train

,0,1,医院欺诈系数1,医院欺诈系数2,单日同一医院最大就诊次数
0,352120001523108,1,0.207418,0.209154,13
1,352120001475556,0,0.053581,0.070967,6
2,352120003484886,0,0.074702,0.072067,3
3,352120002750505,0,0.003731,0.001094,4
4,352120001556755,0,0.000000,0.000000,8
...,...,...,...,...,...
19995,352120001911523,0,0.176705,0.191084,3
19996,352120003184869,0,0.017310,0.012591,2
19997,352120000762238,0,0.041901,0.040705,2
19998,352120002142283,0,0.134112,0.136299,2


### 就诊时间窗口

In [44]:
time_span = []
for i in range(len(df_id_train)):
    #所有时间
    all_times = df_tem.loc[df_id_train[0][i]].index
    #时间转化成mm-dd
    try:
        all_times_new = list((map(lambda x: (int(x.replace(' ','').split('-')[:2][1][:-1]),\
                                             int(x.replace(' ','').split('-')[:2][0])),all_times)))
        #时间排序
        all_times_new.sort()
        #时间跨度
        t_span = (all_times_new[-1][0]-all_times_new[0][0])*30+(all_times_new[-1][1]-all_times_new[0][1])
        time_span.append(t_span)
    except:
        #均值
        time_span.append(166)

In [45]:
df_id_train.insert(5,'时间跨度',time_span)

### 就诊频率

In [46]:
#医院数
df_tem = df_train.groupby('个人编码').count()
df_tem

,顺序号,is_fraud,all,医院编码,药品费发生金额,贵重药品发生金额,中成药费发生金额,中草药费发生金额,药品费自费金额,药品费申报金额,...,住院开始时间,住院终止时间,申报受理时间,出院诊断病种名称,本次审批金额,补助审批金额,医疗救助医院申请,民政救助补助金额,城乡优抚补助金额,操作时间
个人编码,,,,,,,,,,,,,,,,,,,,,
352120000000231,69,69,69,69,69,69,69,69,69,69,...,69,8,8,8,69,0,69,69,69,69
352120000000386,64,64,64,64,64,64,64,64,64,64,...,64,7,7,7,64,0,64,64,64,64
352120000000408,102,102,102,102,102,102,102,102,102,102,...,102,13,13,13,102,0,102,102,102,102
352120000000409,56,56,56,56,56,56,56,56,56,56,...,56,9,9,9,56,0,56,56,56,56
352120000000511,64,64,64,64,64,64,64,64,64,64,...,64,13,13,13,64,0,64,64,64,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352128000611016,85,85,85,85,85,85,85,85,85,85,...,85,10,10,10,85,85,85,85,85,85
352128000613804,89,89,89,89,89,89,89,89,89,89,...,89,10,10,10,89,0,89,89,89,89
352128000614956,131,131,131,131,131,131,131,131,131,131,...,131,16,16,16,131,131,131,131,131,131


In [47]:
frequency = []
for i in range(len(df_id_train)):
    try: 
        num = df_tem.loc[df_id_train[0][i]]['医院编码']
        time_span = df_id_train['时间跨度'][i]
        fre = num/time_span
        if fre != np.inf:
            frequency.append(fre)
        else:
            frequency.append(0)
    except:
        frequency.append(0)

/var/folders/bl/s33lb64n5876ysbhd2h660tw0000gn/T/ipykernel_58632/4170172766.py:6: RuntimeWarning: divide by zero encountered in long_scalars
  fre = num/time_span


In [48]:
df_id_train.insert(6,'就诊频率',frequency)

In [49]:
df_id_train

,0,1,医院欺诈系数1,医院欺诈系数2,单日同一医院最大就诊次数,时间跨度,就诊频率
0,352120001523108,1,0.207418,0.209154,13,180,3.222222
1,352120001475556,0,0.053581,0.070967,6,179,0.284916
2,352120003484886,0,0.074702,0.072067,3,177,0.418079
3,352120002750505,0,0.003731,0.001094,4,178,0.741573
4,352120001556755,0,0.000000,0.000000,8,180,0.883333
...,...,...,...,...,...,...,...
19995,352120001911523,0,0.176705,0.191084,3,0,0.000000
19996,352120003184869,0,0.017310,0.012591,2,0,0.000000
19997,352120000762238,0,0.041901,0.040705,2,0,0.000000
19998,352120002142283,0,0.134112,0.136299,2,0,0.000000


### 金额一次数据

In [50]:
#所有金额部分
columns_all_je = list(filter(lambda x: '金额' in x, df_train.columns))
columns_all_je

['药品费发生金额',
 '贵重药品发生金额',
 '中成药费发生金额',
 '中草药费发生金额',
 '药品费自费金额',
 '药品费申报金额',
 '检查费发生金额',
 '贵重检查费金额',
 '检查费自费金额',
 '检查费申报金额',
 '治疗费发生金额',
 '治疗费自费金额',
 '治疗费申报金额',
 '手术费发生金额',
 '手术费自费金额',
 '手术费申报金额',
 '床位费发生金额',
 '床位费申报金额',
 '医用材料发生金额',
 '高价材料发生金额',
 '医用材料费自费金额',
 '成分输血申报金额',
 '其它发生金额',
 '其它申报金额',
 '一次性医用材料申报金额',
 '起付线标准金额',
 '起付标准以上自负比例金额',
 '医疗救助个人按比例负担金额',
 '最高限额以上金额',
 '基本医疗保险统筹基金支付金额',
 '城乡救助补助金额',
 '可用账户报销金额',
 '基本医疗保险个人账户支付金额',
 '非账户支付金额',
 '本次审批金额',
 '补助审批金额',
 '民政救助补助金额',
 '城乡优抚补助金额']

In [51]:
#涉及金额的一次数据
df_tem = df_train.groupby('个人编码').sum()[columns_all_je]
df_tem.insert(0,0,df_tem.index)

df_tem.reset_index(drop = True, inplace = True)

/var/folders/bl/s33lb64n5876ysbhd2h660tw0000gn/T/ipykernel_58632/894359736.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_tem = df_train.groupby('个人编码').sum()[columns_all_je]


In [52]:
df_id_train = pd.merge(df_tem,df_id_train,on = 0)

In [53]:
df_id_train

,0,药品费发生金额,贵重药品发生金额,中成药费发生金额,中草药费发生金额,药品费自费金额,药品费申报金额,检查费发生金额,贵重检查费金额,检查费自费金额,...,本次审批金额,补助审批金额,民政救助补助金额,城乡优抚补助金额,1,医院欺诈系数1,医院欺诈系数2,单日同一医院最大就诊次数,时间跨度,就诊频率
0,352120000000231,16171.75,1984.60,4776.49,0.00,68.93,15929.04,875.0,0.0,0.0,...,16942.04,0.00,0.00,0.0,0,0.019211,0.027155,3,178,0.387640
1,352120000000386,10698.92,2264.73,1132.45,0.00,31.90,10418.95,345.0,0.0,0.0,...,10915.75,0.00,0.00,0.0,0,0.011356,0.006986,7,166,0.385542
2,352120000000408,14211.17,0.00,3762.15,0.00,4.72,13926.88,0.0,0.0,0.0,...,17840.03,0.00,0.00,0.0,0,0.090030,0.085534,8,173,0.589595
3,352120000000409,6292.38,0.00,5075.85,0.00,1.16,6291.22,0.0,0.0,0.0,...,13718.22,0.00,0.00,0.0,0,0.033455,0.043820,5,179,0.312849
4,352120000000511,14507.54,2367.68,3678.72,0.00,4.59,14402.13,0.0,0.0,0.0,...,14642.13,0.00,0.00,0.0,0,0.179031,0.197765,4,173,0.369942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,352128000611016,11179.38,1391.84,2538.32,0.00,0.77,11123.80,104.5,0.0,0.0,...,11995.70,1854.18,1112.51,0.0,0,0.015738,0.008538,4,174,0.488506
19996,352128000613804,20789.52,0.00,3893.34,6636.17,140.10,20486.62,60.0,0.0,60.0,...,26504.62,0.00,0.00,0.0,0,0.053581,0.070967,6,176,0.505682
19997,352128000614956,29304.95,5925.28,11143.60,786.88,21.50,29103.86,182.0,0.0,0.0,...,32790.36,5316.83,3190.13,0.0,0,0.018914,0.017728,6,174,0.752874
19998,352128000616266,26037.31,21084.46,784.45,0.00,0.02,23928.78,690.0,0.0,0.0,...,24766.48,0.00,0.00,0.0,0,0.090030,0.085534,5,160,0.418750


### 金额二次数据

#### 求和

In [54]:
#发生金额求和
columns_all_fsje = list(filter(lambda x: '发生金额' in x, df_train.columns))
#自费金额求和
columns_all_zfje = list(filter(lambda x: '自费金额' in x, df_train.columns))
#申报金额求和
columns_all_sbje = list(filter(lambda x: '申报金额' in x, df_train.columns))

In [55]:
columns_all_fsje

['药品费发生金额',
 '贵重药品发生金额',
 '中成药费发生金额',
 '中草药费发生金额',
 '检查费发生金额',
 '治疗费发生金额',
 '手术费发生金额',
 '床位费发生金额',
 '医用材料发生金额',
 '高价材料发生金额',
 '其它发生金额']

In [56]:
columns_all_zfje

['药品费自费金额', '检查费自费金额', '治疗费自费金额', '手术费自费金额', '医用材料费自费金额']

In [57]:
columns_all_sbje

['药品费申报金额',
 '检查费申报金额',
 '治疗费申报金额',
 '手术费申报金额',
 '床位费申报金额',
 '成分输血申报金额',
 '其它申报金额',
 '一次性医用材料申报金额']

In [58]:
df_id_train.insert(df_id_train.shape[1],'发生金额求和',df_id_train[columns_all_fsje].sum(axis=1))
df_id_train.insert(df_id_train.shape[1],'自费金额求和',df_id_train[columns_all_zfje].sum(axis=1)) 
df_id_train.insert(df_id_train.shape[1],'申报金额求和',df_id_train[columns_all_sbje].sum(axis=1)) 

#### 求均值

In [59]:
df_id_train.insert(df_id_train.shape[1],'平均发生金额',df_id_train[columns_all_fsje].sum(axis=1)/df_id_train['就诊频率'])
df_id_train.insert(df_id_train.shape[1],'平均自费金额',df_id_train[columns_all_zfje].sum(axis=1)/df_id_train['就诊频率'])
df_id_train.insert(df_id_train.shape[1],'平均申报金额',df_id_train[columns_all_sbje].sum(axis=1)/df_id_train['就诊频率'])

#### 求比例

In [60]:
all_sum = df_id_train[columns_all_fsje].sum(axis=1)+\
          df_id_train[columns_all_zfje].sum(axis=1)+\
          df_id_train[columns_all_sbje].sum(axis=1)

In [61]:
df_id_train.insert(df_id_train.shape[1],'发生金额比例',df_id_train[columns_all_fsje].sum(axis=1)/all_sum)
df_id_train.insert(df_id_train.shape[1],'自费金额比例',df_id_train[columns_all_zfje].sum(axis=1)/all_sum) 
df_id_train.insert(df_id_train.shape[1],'申报金额比例',df_id_train[columns_all_sbje].sum(axis=1)/all_sum) 

#### 极差

In [62]:
columns_all_je

['药品费发生金额',
 '贵重药品发生金额',
 '中成药费发生金额',
 '中草药费发生金额',
 '药品费自费金额',
 '药品费申报金额',
 '检查费发生金额',
 '贵重检查费金额',
 '检查费自费金额',
 '检查费申报金额',
 '治疗费发生金额',
 '治疗费自费金额',
 '治疗费申报金额',
 '手术费发生金额',
 '手术费自费金额',
 '手术费申报金额',
 '床位费发生金额',
 '床位费申报金额',
 '医用材料发生金额',
 '高价材料发生金额',
 '医用材料费自费金额',
 '成分输血申报金额',
 '其它发生金额',
 '其它申报金额',
 '一次性医用材料申报金额',
 '起付线标准金额',
 '起付标准以上自负比例金额',
 '医疗救助个人按比例负担金额',
 '最高限额以上金额',
 '基本医疗保险统筹基金支付金额',
 '城乡救助补助金额',
 '可用账户报销金额',
 '基本医疗保险个人账户支付金额',
 '非账户支付金额',
 '本次审批金额',
 '补助审批金额',
 '民政救助补助金额',
 '城乡优抚补助金额']

In [63]:
columns_all_je_jc = list(map(lambda x :x+'极差',columns_all_je))
columns_all_je_jc

['药品费发生金额极差',
 '贵重药品发生金额极差',
 '中成药费发生金额极差',
 '中草药费发生金额极差',
 '药品费自费金额极差',
 '药品费申报金额极差',
 '检查费发生金额极差',
 '贵重检查费金额极差',
 '检查费自费金额极差',
 '检查费申报金额极差',
 '治疗费发生金额极差',
 '治疗费自费金额极差',
 '治疗费申报金额极差',
 '手术费发生金额极差',
 '手术费自费金额极差',
 '手术费申报金额极差',
 '床位费发生金额极差',
 '床位费申报金额极差',
 '医用材料发生金额极差',
 '高价材料发生金额极差',
 '医用材料费自费金额极差',
 '成分输血申报金额极差',
 '其它发生金额极差',
 '其它申报金额极差',
 '一次性医用材料申报金额极差',
 '起付线标准金额极差',
 '起付标准以上自负比例金额极差',
 '医疗救助个人按比例负担金额极差',
 '最高限额以上金额极差',
 '基本医疗保险统筹基金支付金额极差',
 '城乡救助补助金额极差',
 '可用账户报销金额极差',
 '基本医疗保险个人账户支付金额极差',
 '非账户支付金额极差',
 '本次审批金额极差',
 '补助审批金额极差',
 '民政救助补助金额极差',
 '城乡优抚补助金额极差']

In [64]:
df_tem = df_train.set_index('个人编码')

In [65]:
df_tem.loc[352120001523108][columns_all_je].max(axis = 0)-df_tem.loc[352120001523108][columns_all_je].min(axis = 0)

药品费发生金额           1488.48
贵重药品发生金额           755.28
中成药费发生金额           466.58
中草药费发生金额             0.00
药品费自费金额             32.52
药品费申报金额           1450.78
检查费发生金额            288.00
贵重检查费金额            140.00
检查费自费金额             20.00
检查费申报金额            288.00
治疗费发生金额            460.00
治疗费自费金额              0.00
治疗费申报金额            450.80
手术费发生金额              0.00
手术费自费金额              0.00
手术费申报金额              0.00
床位费发生金额              0.00
床位费申报金额              0.00
医用材料发生金额            11.60
高价材料发生金额             0.00
医用材料费自费金额            6.00
成分输血申报金额             0.00
其它发生金额               0.00
其它申报金额               0.00
一次性医用材料申报金额         11.60
起付线标准金额              0.00
起付标准以上自负比例金额       263.53
医疗救助个人按比例负担金额      290.16
最高限额以上金额             0.00
基本医疗保险统筹基金支付金额    1054.13
城乡救助补助金额           262.29
可用账户报销金额            65.57
基本医疗保险个人账户支付金额      39.20
非账户支付金额             65.57
本次审批金额            1450.78
补助审批金额             327.86
民政救助补助金额           262.29
城乡优抚补助金额             0.00
dtype: float

In [66]:
df_new = pd.DataFrame(index=df_id_train.index,columns=columns_all_je)

In [67]:
for i in range(len(df_new)):
    id_ = df_id_train[0][i]
    df_new.loc[i,:] = df_tem.loc[id_][columns_all_je].max(axis = 0)-df_tem.loc[id_][columns_all_je].min(axis = 0)

In [68]:
df_new.columns = columns_all_je_jc

In [69]:
df_id_train = pd.concat([df_id_train,df_new],axis=1)

In [81]:
df_id_train.rename(columns={0:'个人编号',1:'是否欺诈'},inplace=True)

In [99]:
df_id_train.replace(np.nan,0,inplace=True)
df_id_train.replace(np.inf,0,inplace=True)

In [ ]:
df_id_train.to_excel('../result/features.xlsx')

## 归一化

In [100]:
X = df_id_train.drop(columns=['个人编号','是否欺诈']).values
y = df_id_train['是否欺诈'].values

In [101]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [102]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
std = StandardScaler()
X_train_std = std.fit_transform(X_train)
X_test_std = std.transform(X_test)